In [1]:
import ROOT
import numpy as np
import pandas as pd
%jsroot on

Welcome to JupyROOT 6.19/01


In [2]:
def convert_th2fs_to_dataframe(file_name, dir_name):
    
    columns=['name', 'image']
    df  = pd.DataFrame(columns=columns)
    file = ROOT.TFile(file_name)
    file.cd(dir_name)
    total_energy = []
    
    for key in ROOT.gDirectory.GetListOfKeys():
        image = []
        if key.GetName() != "Info":
            calo_image_name = key.GetName()
            calo_image      = file.FindObjectAny(calo_image_name)
            calo_image.Scale(0.001)
            #print(calo_image_name)
            for i in range(0, calo_image.GetNbinsX()):
                row   = []
                for j in range(0, calo_image.GetNbinsY()):
                    pix = float(calo_image.GetBinContent(i+1,j+1))
                    row.append(pix)
                image.append(row)
            #image = np.array(image)
            #print(image)
            sum = 0
            for row in image:
                sum += np.sum(row)
            
            if(sum > 100):
                df2 = pd.DataFrame([[calo_image_name, image]], columns=columns)
                df = df.append(df2, ignore_index=True)
    return df

In [3]:
data = convert_th2fs_to_dataframe("wjets_merged.root", "DQMData")

In [4]:
data.to_hdf("calo_images.h5", key="data", mode="w")

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['name', 'image']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
